#### Notebook for UAS TIR imagery analysis

In [ ]:
import os
import numpy as np
from numpy import genfromtxt
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from scipy import stats
import cv2

%matplotlib inline

***
***
#### Functions:

In [ ]:
def linfit_conf(x,y,alpha,sided=2):
    '''compute confidence intervals for a linear regression 
    line given an alpha and one or two sided test'''
    # Modified code from: https://tomholderness.wordpress.com/2013/01/10/confidence_intervals/
    # linfit.py - example of confidence limit calculation for linear regression fitting.
    
    # References:
    # - Statistics in Geography by David Ebdon (ISBN: 978-0631136880)
    # - Reliability Engineering Resource Website:
    # - http://www.weibull.com/DOEWeb/confidence_intervals_in_simple_linear_regression.htm
    # - University of Glascow, Department of Statistics:
    # - http://www.stats.gla.ac.uk/steps/glossary/confidence_intervals.html#conflim

    x = np.array(x)
    y = np.array(y)
    
    # fit a line to the data using a least squares 1st order polynomial fit
    z = np.polyfit(x,y,1)
    p = np.poly1d(z)
    fit = p(x)

    # get the coordinates for the fit line
    c_y = [np.min(fit),np.max(fit)]
    c_x = [np.min(x),np.max(x)]
 
    # predict y values of origional data using the fit
    p_y = z[0] * x + z[1]

    # calculate the y-error (residuals)
    y_err = y - p_y
 
    # create series of new test x-values to predict for
    p_x = np.linspace(np.min(x),np.max(x),30)
 
    # now calculate confidence intervals for new test x-series
    mean_x = np.mean(x)                     # mean of x
    n = len(x)                              # number of samples in origional fit
    t = stats.t.ppf(1-(alpha/sided), n-1)   # find t value (1 or 2 sided)
    s_err = np.sum(np.power(y_err,2))       # sum of the squares of the residuals
 
    confs = t * np.sqrt((s_err/(n-2))*(1.0/n + (np.power((p_x-mean_x),2)/
                ((np.sum(np.power(x,2)))-n*(np.power(mean_x,2))))))
 
    # now predict y based on test x-values
    p_y = z[0] * p_x + z[1]
 
    # get lower and upper confidence limits based on predicted y and confidence intervals
    lower = p_y - abs(confs)
    upper = p_y + abs(confs)
 
    return [p_x, lower, upper]

def quick_lin(x,y,color='r',alpha=0.01,print_output=True):
    '''computes and plots a simple linear regression line
    and tests for statistical significance, using SciPy'''
    
    ## Pearson's r (parametric correlation test):
    pearsons_r = stats.pearsonr(x, y)
    
    ## Kendall's tau (non-parametric correlation test):
    kendalls_tau = stats.kendalltau(x, y)
    
    # Linear Regression (parametric test for slope = 0):  
    # https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.linregress.html
    # performs a Wald Test against null hypothesis that slope is zero -- https://en.wikipedia.org/wiki/Wald_test
    # See also: https://github.com/scipy/scipy/issues/7074
    slope, intercept, r, p, SE = stats.linregress(x,y)
    r2 = r**2
    
    # null hypothesis: slope (dy/dx) is zero, that is, y does not change as a function of x
    slope0 = 0; 
    # alternative hypothesis: slope is non-zero (either positive or negative, two-tailed test)
    # Student's t-test (parametric test): https://en.wikipedia.org/wiki/Student%27s_t-test
    t_value = ((slope - (slope0))/SE) # SE = standard error
    n = len(x)
    # P-value from Student's t-test
    p_value = stats.t.sf(np.abs(t_value), n-1)*2
    # Reject the null hypothesis if p_value <= alpha
      
    # confidence intervals
    alpha = 0.05
    [p_x, lower, upper] = linfit_conf(x,y,alpha)
        
    # Plot the linear fit
    x_lin = np.linspace(min(x),max(x),n)
    plt.plot(x_lin, slope*x_lin + intercept, ':', c=color, label='Linear Regression')
    plt.fill_between(p_x,lower,upper,color=color,alpha=0.1,label='95% Confidence')
    #plt.legend()
    
    # calculate the residuals
    residuals = y - (slope*x_lin + intercept) # residuals = measured - model
    
    # print output if flagged True
    if print_output == True:
        print("Linear regression:\n\tslope=%s\n\tintercept=%s\n\tr=%s\n\tr2=%s\n\tp-value=%s\n\tSE=%s" \
          % (str(slope), str(intercept), str(r), str(r2), str(p), str(SE)))
        print("Student's t-test: \n\tp-value=%s" % str(p_value))
        print("Pearson's r:\n\tcorr_coeff=%s\n\tp-value=%s" \
          % (str(pearsons_r[0]), str(pearsons_r[1])))
        print("Kendalls tau:\n\t%s" % str(kendalls_tau))
    
    return slope, r, p, SE, t_value, p_value, residuals


In [ ]:
def count_px(image):
    '''counts the number of pixels in an image (or the number of elements in any n-D array)'''
    n_pixels = 1
    for dim in np.shape(image): 
        n_pixels *= dim
    return n_pixels

def rmse(value, predictions):
    '''root mean square error'''
    error = predictions - value
    SE = error**2
    MSE = np.mean(SE)
    RMSE = np.sqrt(MSE)
    mean_bias = np.mean(error) # mean of (predictions - value)
    return RMSE, mean_bias

In [ ]:
def create_histogram(image, n_bins=None, low_threshold=None, high_threshold=None):
    '''creates a single image's histogram  '''
    # reshape the image pixel array
    image = image.reshape(-1);
    # remove NaN values
    image = image[~np.isnan(image)];
    # apply low_threshold:
    if low_threshold is not None:
        image = image[image>low_threshold]
    if high_threshold is not None:
        image = image[image<high_threshold]
    # create the histogram:
    if n_bins is None:
        n_bins = int(np.round(max(image) - min(image),0)) # bin width should be set to accuracy of the camera? +/- 1 C
    count, value = np.histogram(image, bins=n_bins); 
    # take the bin edges (value) and change them to bin centers
    # so that number of "count" = number of "value"
    new_value = [(i+value[idx+1])/2 for idx, i in enumerate(value) if idx < len(value)-1]
    value = new_value
    count_stddev = np.std(count)
    return [value, count]



In [ ]:
def radiometric_correction(data, image, Tss):
    '''Radiometric correction using a known snow surface temperature (Tss)
       Corrects some original array (data) using a histogram based on a 
       subset (image) and known, uniform, snow surface temperature (Tss)'''
    # if data == image then the radiometric correction is performed based on the entire data array
    image = image[~np.isnan(image)]; # this should take care of the flattening of the array itself
    # find peaks (modes) in the histogram:
    value, count = create_histogram(image,int(np.sqrt(len(image)))) #255
    count_stddev = np.std(count)
    # identify which one is the the "snow peak" -- assumed to be the coldest peak in the image's histogram
    modes = []
    counts = []
    if sagehen == True:
        for i in range(0,len(count)-1):
            if count[i]-np.mean(count) > count_stddev:
                # if the difference between the peak the local mean is > 1 sigma, then we'll call this a peak
                if i > 0:
                    if count[i-1] < count[i] and count[i+1] < count[i]:
                        # if the count at this index is a local maximum based on its two neighbors
                        #print('peak at i=',i)
                        modes.append(value[i])
                        counts.append(count[i])
                else:
                    if count[i+1] < count[i]:
                        # if the count at this index (0th index) is a local maximum based on its one neighbor
                        #print('peak at i=',i)
                        modes.append(value[i])
                        counts.append(count[i])
    if sagehen == False:
        for i in range(1,len(count)-1):
            if count[i-1] < count[i] and count[i+1] < count[i]:
                # if the count at this index is a local maximum based on its two neighbors
                #if count[i]-np.mean(count) > count_stddev: # and if the difference between the peak the local mean is > 1 sigma, then we'll call this a peak (used for Sagehen)
                if value[i] < 0: # and if the peak appears colder than 0 C (using this for Davos right now)    
                    #print('peak at i=',i)
                    modes.append(value[i])
                    counts.append(count[i])

    #print('snow peak:', modes[0]) # show us what the coldest peak in the histogram is
    
    # set the radiometric ("histogram") offset to this minimum peak,
    # so that the peak will be equal to own known value of Tss
    #print(modes)
    hist_offset = modes[0] - Tss

    # subtract the radiometric offset value from the entire image data
    data_corrected = data - hist_offset; 
    
    return data_corrected, hist_offset, modes[0]

In [ ]:
def getSummaryStats(image):
    '''compute some stats for a given image'''
    i_mean = np.mean(image)
    i_median = np.median(image)
    i_std = np.std(image)
    i_max = np.max(image)
    i_min = np.min(image)
    i_range = i_max - i_min
    
    return [i_mean, i_median, i_std, i_max, i_min, i_range]

In [ ]:
def upscaling_routines(original_image,px_size,Tss,target_res,make_plots=False,save_plots=False,plot_index=0):
    ### Perform the radiometric correction on the original image:
    image = original_image.reshape(-1);
    ### Perform the radiometric correction of "data[:,:,img]"" based on the histogram of "image", and "Tss"
    image_corrected = radiometric_correction(original_image,image,Tss)
    
    ### Upscale the image_corrected
    # Gaussian Pyramids with OpenCV:
    steps = int(np.round(np.log(target_res/px_size)/(np.log(2)))) # how many steps to take to get (close) to target resolution
    g_new_res = px_size * 2**steps # what the actual new resolution will be
    lower_res = image_corrected # start with the original image, each iteration makes it lower resolution
    for i in range(0,steps):
        lower_res = cv2.pyrDown(lower_res) # iterate using pyrDown the number of steps we need
    gaussian_upscaled = lower_res 
    
    
    ### Compare some stats from the original image to the upscaled image(s)
    # Original image:
    [o_mean, o_median, o_std, o_max, o_min] = getSummaryStats(image_corrected)
    o_text = "Original Image:\n$\overline{T}$: %.2f \n$\widetilde{T}$: %.2f \n$\sigma_{T}$: %.2f \n$T_{MAX}$: %.2f \n$T_{MIN}$: %.2f" % (o_mean, o_median, o_std, o_max, o_min)
    # Gaussian upscaled image:
    [g_mean, g_median, g_std, g_max, g_min] = getSummaryStats(gaussian_upscaled)
    g_text = "Gaussian Upscaled:\n$\overline{T}$: %.2f \n$\widetilde{T}$: %.2f \n$\sigma_{T}$: %.2f \n$T_{MAX}$: %.2f \n$T_{MIN}$: %.2f" % (g_mean, g_median, g_std, g_max, g_min)
    # Simple mean aggregated image:
    [m_mean, m_median, m_std, m_max, m_min] = getSummaryStats(mean_agg)
    m_text = "Simple Mean:\n$\overline{T}$: %.2f \n$\widetilde{T}$: %.2f \n$\sigma_{T}$: %.2f \n$T_{MAX}$: %.2f \n$T_{MIN}$: %.2f" % (m_mean, m_median, m_std, m_max, m_min)
    # Spatial average aggregated image:
    [s_mean, s_median, s_std, s_max, s_min] = getSummaryStats(spatial_average)
    s_text = "Spatial Mean:\n$\overline{T}$: %.2f \n$\widetilde{T}$: %.2f \n$\sigma_{T}$: %.2f \n$T_{MAX}$: %.2f \n$T_{MIN}$: %.2f" % (s_mean, s_median, s_std, s_max, s_min)
    
    ### Compare some stats from the upscaled image(s) to aircraft imagery
    
    ### Plots
    if make_plots == True:
        ### Plot images
        plt.figure(figsize=(20,5))
        
        v_min=0
        v_max=25
        
        plt.subplot(141)
        plt.imshow(image_corrected,vmin=v_min,vmax=v_max,cmap='magma')
        plt.title("original image: " + str(np.round(px_size,2)))
        
        plt.subplot(142)
        plt.imshow(gaussian_upscaled,vmin=v_min,vmax=v_max,cmap='magma')
        plt.title("pyrDown upscaled image: " + str(np.round(g_new_res,2)))
        
        plt.subplot(143)
        plt.imshow(mean_agg,vmin=v_min,vmax=v_max,cmap='magma')
        plt.title("mean aggregated image: " + str(np.round(m_new_res,2)))
        
        plt.subplot(144)
        plt.imshow(spatial_average,vmin=v_min,vmax=v_max,cmap='magma')
        plt.title("spatial average image: " + str(np.round(m_new_res,2)))
        

        if save_plots == True:
            plt.savefig(r'plots\upscaling\\images_' + str(plot_index) + '.png')
        
        ### T vs T plots
        plt.figure(figsize=(20,5))
               
        plt.subplot(141)
        compareResDeltaHist(image_corrected,image_corrected,lims=(-5,35))
        plt.title("original image: " + str(np.round(px_size,2)))
        
        plt.subplot(142)
        compareResDeltaHist(image_corrected,gaussian_upscaled,lims=(-5,35))
        plt.title("pyrDown upscaled image: " + str(np.round(g_new_res,2)))
        
        plt.subplot(143)
        compareResDeltaHist(image_corrected,mean_agg,lims=(-5,40))
        plt.title("mean aggregated image: " + str(np.round(m_new_res,2)))
        
        plt.subplot(144)
        compareResDeltaHist(image_corrected,spatial_average,lims=(-5,35))
        plt.title("spatial average image: " + str(np.round(m_new_res,2)))
        
        if save_plots == True:
            plt.savefig(r'plots\upscaling\\OriginalTvsDeltaT_hist_' + str(plot_index) + '.png')
        
        
        
        ### Histogram plots
        plt.figure(figsize=(20,5))
        original_hist = create_histogram(image_corrected)
        plt.plot(original_hist[0],original_hist[1]/np.sum(original_hist[1]),'--k',label='original')
        gaussian_hist = create_histogram(gaussian_upscaled)
        plt.plot(gaussian_hist[0],gaussian_hist[1]/np.sum(gaussian_hist[1]),':k',label='gaussian pyramid')
        mean_agg_hist = create_histogram(mean_agg)
        plt.plot(mean_agg_hist[0],mean_agg_hist[1]/np.sum(mean_agg_hist[1]),':r',label='simple average')
        spatial_avg_hist = create_histogram(spatial_average)
        plt.plot(spatial_avg_hist[0],spatial_avg_hist[1]/np.sum(spatial_avg_hist[1]),':b',label='spatial average')
        
        plt.text(-5,0.25,o_text)
        plt.text(5,0.25,g_text)
        plt.text(15,0.25,m_text)
        plt.text(25,0.25,s_text)
        
        plt.ylim((0,0.4))
        plt.xlim((-10,40))
        
        plt.legend()
        if save_plots == True:
            plt.savefig(r'plots\upscaling\\histogram_' + str(plot_index) + '.png')
    
    return None

In [ ]:
# Three different plot types for comparing an image that's been scaled to a different resolution with its original

def compareResScatter(original_img,scaled_img,lims=(-5,35)):
    zoom = np.max([original_img.shape[0]/scaled_img.shape[0],original_img.shape[1]/scaled_img.shape[1]])
    scaled_original_res = ndimage.zoom(scaled_img, zoom, order=0, mode='constant', cval=0.0, prefilter=True)
    scaled_original_res = scaled_original_res[0:original_img.shape[0],0:original_img.shape[1]]
    plt.scatter(scaled_original_res,original_img,c='k',s=0.1,alpha=0.1)
    # add 1:1 line
    plt.plot(np.linspace(lims[0],lims[1],10),np.linspace(lims[0],lims[1],10),':b')
    # set axes limits
    plt.ylim(lims)
    plt.xlim(lims)
    plt.ylabel("Original T")
    plt.xlabel("Scaled T")
    return None


def compareResDelta(original_img,scaled_img,lims=(-5,40)):
    zoom = np.max([original_img.shape[0]/scaled_img.shape[0],original_img.shape[1]/scaled_img.shape[1]])
    scaled_original_res = ndimage.zoom(scaled_img, zoom, order=0, mode='constant', cval=0.0, prefilter=True)
    scaled_original_res = scaled_original_res[0:original_img.shape[0],0:original_img.shape[1]]
    plt.scatter(original_img,scaled_original_res-original_img,c='k',s=0.1,alpha=0.1)
    # add 1:1 line
    plt.plot(np.linspace(lims[0],lims[1],10),np.linspace(0,0,10),':b')
    # set axes limits
    plt.ylim((-30,30))
    plt.xlim(lims)
    plt.ylabel("Scaled T - Original T")
    plt.xlabel("Original T")
    return None


def compareResDeltaHist(original_img,scaled_img,lims=(-20,20)):
    zoom = np.max([original_img.shape[0]/scaled_img.shape[0],original_img.shape[1]/scaled_img.shape[1]])
    scaled_original_res = ndimage.zoom(scaled_img, zoom, order=0, mode='constant', cval=0.0, prefilter=True)
    scaled_original_res = scaled_original_res[0:original_img.shape[0],0:original_img.shape[1]]
    # create and plot histogram
    n_bins = 500
    count, value = np.histogram(scaled_original_res-original_img, bins=n_bins); 
    # take the bin edges (value) and change them to bin centers
    # so that number of "count" = number of "value"
    new_value = [(i+value[idx+1])/2 for idx, i in enumerate(value) if idx < len(value)-1]
    value = new_value
    plt.plot(value,count,'-k')
    # add 0 line
    plt.plot(np.linspace(0,0,10),np.linspace(0,10000,10),':b')
    # set axes limits
    plt.ylim((0,20000))
    plt.xlim((-20,20))
    plt.ylabel("Pixel Count")
    plt.xlabel("Scaled T - Original T")
    return None



#### Simple test for significant change in mean, standard deviation, or range of T:

In [ ]:
def test_for_T_change(px_sizes,means,stddevs,mins,maxs,std_error_mean,std_error,color='k',save_figure=False):
    '''Test for significant change in temperature mean, standard deviation, or range of specified class'''
    print("\nMean")
    plt.figure(0,figsize=(6,6))
    plt.plot(px_sizes,means,'.',c=color, label='$Mean T_{}$',);
    plt.errorbar(px_sizes,means, fmt='none', xerr=None, yerr=std_error_mean, ecolor=color, elinewidth=0.5, capsize=None)
    result = quick_lin(px_sizes,means,color)
    plt.ylabel('$Mean T_{}, C^{O}$');
    plt.xlabel('Pixel Size, m')
    textstr = '$R^2$=%.2f\np-value=%e' % (result[1]**2, result[2])
    plt.title('$Mean T_{}$, n=%d\n%s' % (n, textstr));
    #props = dict(boxstyle='square', facecolor='lightgrey', alpha=0.5)
    #plt.text(1, 0.045, textstr, fontsize=12, \
    #        verticalalignment='top', bbox=props)
    mean_residuals = result[6]
    ###
    plt.xlim((0,0.2))
    plt.xticks(np.arange(0, 0.21, 0.05))
    plt.ylim((0,16))
    ###
    if save_figure==True:
        plt.savefig(r'plots\PaperFigures\meanfig.png',
                transparent=True, dpi=300)
    ###
    print("Change in mean (C):")
    print((means[-1]-means[0]))
    print("Change in  mean (%):")
    print(((means[-1]-means[0])/means[0])*100)


    print("\nStandard Deviation")
    plt.figure(1,figsize=(5,5))
    plt.plot(px_sizes,stddevs,'.k', label='$\sigma\ T_{}$',);
    plt.errorbar(px_sizes,stddevs, fmt='none', xerr=None, yerr=std_error, ecolor='k', elinewidth=0.5, capsize=None)
    result = quick_lin(px_sizes,stddevs)
    plt.ylabel('$T_{} Standard Deviation, C^{O}$');
    plt.xlabel('Pixel Size, m')
    textstr = '$R^2$=%.2f\np-value=%e' % (result[1]**2, result[2])
    plt.title('Standard Deviation $T_{}$, n=%d\n%s' % (n, textstr));
    #props = dict(boxstyle='square', facecolor='lightgrey', alpha=0.5)
    #plt.text(1.5, 0.05, textstr, fontsize=12, \
    #        verticalalignment='top', bbox=props)
    std_residuals = result[5]
    print("Change in stddevs (C):")
    print((stddevs[-1]-stddevs[0]))
    print("Change in  stddevs (%):")
    print(((stddevs[-1]-stddevs[0])/stddevs[0])*100)

    print("\nMin")
    plt.figure(2,figsize=(5,5))
    plt.plot(px_sizes,mins,'.k', label='Min $T_{}$',);
    plt.errorbar(px_sizes,mins, fmt='none', xerr=None, yerr=std_error, ecolor='k', elinewidth=0.5, capsize=None)
    result = quick_lin(px_sizes,mins)
    plt.ylabel('$Min T_{}, C^{O}$');
    plt.xlabel('Pixel Size, m')
    textstr = '$R^2$=%.2f\np-value=%e' % (result[1]**2, result[2])
    plt.title('Min $T_{}$, n=%d\n%s' % (n, textstr));
    #props = dict(boxstyle='square', facecolor='lightgrey', alpha=0.5)
    #plt.text(1.5, 0.05, textstr, fontsize=12, \
    #        verticalalignment='top', bbox=props)
    min_residuals = result[5]
    
    print("\nMax")
    plt.figure(3,figsize=(5,5))
    plt.plot(px_sizes,maxs,'.k', label='Min $T_{}$',);
    plt.errorbar(px_sizes,maxs, fmt='none', xerr=None, yerr=std_error, ecolor='k', elinewidth=0.5, capsize=None)
    result = quick_lin(px_sizes,maxs)
    plt.ylabel('$Max T_{}, C^{O}$');
    plt.xlabel('Pixel Size, m')
    textstr = '$R^2$=%.2f\np-value=%e' % (result[1]**2, result[2])
    plt.title('Max $T_{}$, n=%d\n%s' % (n, textstr));
    #props = dict(boxstyle='square', facecolor='lightgrey', alpha=0.5)
    #plt.text(1.5, 0.05, textstr, fontsize=12, \
    #        verticalalignment='top', bbox=props)
    max_residuals = result[5]

In [ ]:
# Test sensitivity to measurement error
def msmtErrorTest(x,y,s,m=100):
    # error (s) should be in deg C, standard deviaion of the normally distributed error
       
    # create data with normally distributed random error around the mean value
    y_m = np.ones((m,len(y))) # create array to hold new x values for each of m test runs
    slopes = []; r2s = []; ps = []; ps2 = []; # empty array for slope, r, p results from linear regressions
    for i in range(0,m):
        y_m[i] = np.random.normal(y,s)
        plt.plot(x,y_m[i],'sk',alpha=(m*0.1)**-1);
        # perform a linear regression on each set to test for significance of slope different than 0
        [slope, r, p, SE, t_value, p_value, residuals] = quick_lin(x,y_m[i],'r',0.05,False)
        slopes.append(slope);
        r2s.append(r**2);
        ps.append(p);
        ps2.append(p_value);
        
    # calculate the 95% confidence intervals around each mean value using the test runs above
    p95 = np.ones((len(y),2)) # create array to hold the upper and lower 95% confidence intervals for each y value
    errorbars = np.ones((len(y),2)) # create array to hold the upper and lower errorbar magnitudes
    for i in range (0,len(y)):
        conf_upper = np.percentile(y_m[:][i], 97.5)
        conf_lower = np.percentile(y_m[:][i], 2.5)
        p95[i] = [conf_upper, conf_lower]
        errorbars[i] = [conf_upper-y[i], y[i]-conf_lower]
    
    return y_m, p95, errorbars, slopes, r2s, ps, ps2


# Plot the sensitivity to measurement error tests
def msmtErrorTest_plots(explanatory_variable,test_variable,s,m=100):

    plt.figure(0,figsize=(7,3))
    [y_tests, p95, errorbars, slopes, r2s, ps, ps2] = msmtErrorTest(explanatory_variable,test_variable,s,m)
    plt.plot(explanatory_variable,test_variable,'ob');
    plt.errorbar(explanatory_variable,test_variable, fmt='none', xerr=None, yerr=[errorbars.T[1], errorbars.T[0]], ecolor='b', elinewidth=1, capsize=1)
    #plt.plot(px_sizes,p95.T[0],'or')
    #plt.plot(px_sizes,p95.T[1],'om')
    plt.title('Linear Regression Monte Carlo Tests')
    plt.xlabel('explanatory_variable')
    plt.ylabel('test_variable')
    plt.tight_layout()

    bins = int(np.sqrt(m))

    plt.figure(1,figsize=(7,7))
    plt.subplot(221)
    plt.hist(slopes,bins)
    plt.plot(np.linspace(0, 0),np.linspace(0,m*.1),':k',label='slope=0')
    plt.title('Slope')
    plt.xlabel('degrees C / meter (pixel resolution)')
    plt.ylabel('number of occurrences')
    plt.legend()

    plt.subplot(222)
    plt.hist(r2s,bins)
    plt.title('Coefficient of Determination (r2)')
    plt.xlabel('coefficient of determination (r2)')
    plt.ylabel('number of occurrences')

    # If i'm doing a 2-sided test with alpha=5%, then reject the null (no slope) if p <= alpha/2
    a_2 = 0.025

    plt.subplot(223)
    plt.hist(ps,bins)
    plt.plot(np.linspace(a_2, a_2),np.linspace(0,m*.25),':k',label='alpha')
    plt.legend()
    plt.title('Wald Test p-value')
    plt.xlabel('p-value')
    plt.ylabel('number of occurrences')

    plt.subplot(224)
    plt.hist(ps2,bins)
    plt.plot(np.linspace(a_2, a_2),np.linspace(0,m*.25),':k',label='alpha')
    plt.legend()
    plt.title('Students t-test p-value')
    plt.xlabel('p-value')
    plt.ylabel('number of occurrences')
    plt.tight_layout()
    return None



In [ ]:
def cleanhist(x, n_bins=None,lower_limit=-2,upper_limit=25):
    if n_bins is None:
        n_bins = int(np.round(np.max(x) - np.min(x),0)) # bin width default to +/- 1
    count, value = np.histogram(x, bins=n_bins); 
    # take the bin edges (value) and change them to bin centers
    # so that number of "count" = number of "value"
    new_value = [(i+value[idx+1])/2 for idx, i in enumerate(value) if idx < len(value)-1]
    value = new_value
       
        
    median_x = np.median(x)
    mean_x = np.mean(x)
    mode_x = value[count.argmax()]  
    
    print('Mean:\t%f\nMedian:\t%f\nMode:\t%f' % (mean_x, median_x, mode_x))
        
    
    fig = plt.figure(figsize=(5,5),frameon=False)
    ax = fig.add_subplot(1, 1, 1)
    #plt.hist(x,bins=n_bins,label='Temperature Histogram',color='lightgray');
    ax.fill_between(value,count,0,color='0.5',alpha=0.1)
    ax.plot(np.linspace(0,0,10),np.linspace(0,np.max(count)+1,10),'-',c='lightgray',label='0 C', linewidth=1)
    ax.plot(value,count,'-k', linewidth=1)
    #ax.plot(np.linspace(median_x,median_x,10),np.linspace(0,15,10),'--k',label='Median Temp.')
    #ax.plot(np.linspace(mean_x,mean_x,10),np.linspace(0,15,10),'-k',label='Mean Temp.')
    #ax.plot(np.linspace(np.min(value),np.max(value),10),np.linspace(0,0,10),':k',label='0 C')
    #ax.text(mean_x,14,str(np.round(mean_x,2)))
    #ax.text(median_x,12,str(np.round(median_x,2)))
    plt.xlabel('Temperature (C)')
    plt.ylabel('Number of Samples')
    ax.xaxis.grid(which='major', color='lightgray', linestyle=':', linewidth=0.7)
    
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    #ax.spines['bottom'].set_visible(False)
    #ax.spines['left'].set_visible(False)
    plt.ylim((0,np.max(count)+1))
    plt.xlim((lower_limit,upper_limit))
    plt.tight_layout()
    #plt.legend()

***
***

### Load IR images:

In [ ]:
# Path to directory for the vertical flight TIR image csv files

# Sagehen:
sagehen = True;
path = r'data\ici_csvs\vertical_day1'


# Davos:
#sagehen = False;
#path = r'DFlights_20170327\Flight_1\Thermal with Heights'
#path = r'Flights_20170327\Flight_2\Thermal with Heights'


In [ ]:
# Read in the UAS TIR images

# file extension to detect in this directory
extension = '.csv'

# load all the TIR image files into an array (called "data")
print("Loading files...\n")
file = 1;
for root, dirs_list, files_list in os.walk(path):
    for file_name in files_list:
        if os.path.splitext(file_name)[-1] == extension:
            file_name_path = os.path.join(root, file_name)
            #print(file_name)
            print(file_name_path)   # This is the full path of the file
            if sagehen == True: d = ','
            if sagehen == False: d = ';'
            current_image = genfromtxt(file_name_path, delimiter=d)
            if sagehen == True: #only do this for sagehen images
                # remove the last column of NaN values that genfromtxt is pulling in from some of the csv files
                original_shape = (512,640)
                #current_image = current_image[:,0:-1]
                current_image = current_image[~np.isnan(current_image)]
                current_image = np.reshape(current_image,original_shape)
            if sagehen == False: # check to see if I also need to remove nans from the Davos imagery
                original_shape= (288, 382)
                current_image = current_image[~np.isnan(current_image)]
                current_image = np.reshape(current_image,original_shape)
            if file == 1:
                # if this is the first image, then create the data array with the first image
                data = current_image
            else:
                # if this is not the first image, add this new image to the array containing the previous ones
                data = np.dstack((data,current_image))
            file = file + 1; # increment the file counter

nfiles = file
print(nfiles)
            
original_data = np.copy(data); # keep a copy of the unaltered data for later comparison


### Load corresponding flight log data:

In [ ]:
if sagehen == True:
    # Sagehen:
    # Load data from flight log file: verticalCamTriggerInfo.csv
    flight_log_file = r'data\auxilary\verticalCamTriggerInfo.csv'
    # lambda function for reading datestamps
    str2date = lambda x: dt.strptime(x.decode("utf-8"), '%Y-%m-%d %H:%M:%S.SSS')
    # read the file
    flight_log = genfromtxt(flight_log_file, delimiter=',', skip_header=0, names=True, converters = {'datetime': str2date})
    
if sagehen == False:
    # Davos:
    # Load data from flight altitude log file:
    flight_log_file = r'Flights_20170327\Flight_1\Thermal with Heights\Flights_20170327_Flight1_Heights_70images.txt'
    #flight_log_file = r'Flights_20170327\Flight_2\Thermal with Heights\Flights_20170327_Flight2_Heights_70images.txt'
    # read the file
    flight_log = genfromtxt(flight_log_file, dtype=[('Filename', '<S30'), ('Height_above_reference_m', 'i8')], delimiter='\t', skip_header=0, names=True)

print("Loaded:")
print(flight_log_file)


### Retrieve flight altitudes and set camera geometry variables:

In [ ]:
# Get flight altitude for each image (in different columns for the two data sets, specified with "c")
if sagehen == True: c = 9;
if sagehen == False: c = 1;
img_heights = [];
for i in range(0,data.shape[2]):
    img_heights.append(flight_log[i][c]);


# camera FOV (degrees) and image dimensions (pixels):
if sagehen == True:
    fov_x = 50
    fov_y = 37.5
    pix_x = 640
    pix_y = 512
if sagehen == False:
    fov_x = 38
    fov_y = 29
    pix_x = 382
    pix_y = 288

### Define area of interest (AOI) bounding boxes around target tree in each image:

In [ ]:
# Investigate temperature distributions across selected areas of interest (AOI)
# Each AOI defined as a rectangle in each image: [top left pixel X, top teft pixel Y, width, height]
# Images corresponding with the AOIs are specified by image indices in "images"

# Sagehen:
sagehen_images = list(range(5,46,1))
sagehen_aoi = [
    [33,270,180,180], [20,270,180,180], [80,260,140,140], [70,270,140,140], [60,275,137,137], [133,260,107,107], 
    [133,250,107,107], [138,250,107,107], [160,188,95,95], [175,180,97,97], [180,177,97,97], [190,168,87,87], 
    [190,158,85,85], [195,154,86,86], [180,160,82,82], [220,205,82,82], [160,112,78,78], [510,140,86,86], 
    [507,150,87,87], [499,159,86,86], [483,155,85,85], [480,140,82,82], [479,160,74,74], [488,187,76,76], 
    [502,180,75,75], [486,135,79,79], [481,149,79,79], [458,155,69,69], [470,160,68,68], [469,140,68,68], 
    [477,162,68,68], [458,163,67,67], [464,163,60,60], [468,166,60,60], [453,172,60,60], [465,170,60,60],
    [455,165,57,57], [448,165,55,55], [452,171,55,55], [442,172,55,55], [444,165,55,55]
]

# AOI for flight 1:
davos_images_1 = list(range(0,nfiles-1))
davos_aoi_1 = [
    [75,0,307,287], [75,20,290,290], [75,30,270,270], [75,30,270,270], [75,30,270,270], [75,45,265,265], [78,40,250,250], 
    [78,45,240,240], [78,45,235,235], [80,45,230,230], [80,45,225,225], [80,45,225,225], [88,45,210,210], 
    [90,50,200,200], [100,60,195,195], [100,60,190,190], [100,60,185,185], [105,60,180,180], [110,60,180,180], 
    [105,60,175,175], [105,60,175,175], [105,75,175,175], [100,75,175,175], [100,75,175,175], [105,75,170,170], 
    [110,75,160,160], [110,75,150,150], [115,75,145,145], [115,75,140,140], [115,80,140,140], [125,75,135,135], 
    [125,75,135,135], [125,75,135,135], [125,75,135,135], [125,80,135,135], [125,75,130,130], [125,75,130,130], 
    [125,80,130,130], [125,80,130,130], [130,85,125,125], [130,85,120,120], [135,85,115,115], [130,85,115,115], 
    [135,85,110,110], [135,80,110,110], [130,85,110,110], [130,85,110,110], [130,85,110,110], [135,85,110,110], 
    [140,90,105,105], [140,85,105,105], [140,85,105,105], [140,90,105,105], [145,90,100,100], [145,90,100,100], 
    [145,90,100,100], [145,90,95,95], [150,95,90,90], [150,95,90,90], [150,100,90,90], [150,100,90,90], 
    [150,100,90,90], [150,100,85,85], [150,100,85,85], [150,100,85,85], [155,100,85,85], [155,100,80,80], 
    [155,100,80,80], [160,100,80,80], [160,100,80,80]
]



# AOI for flight 2:
davos_images_2 = list(range(0,nfiles-1))
davos_aoi_2 = [
    [175,60,50,80], [175,60,50,80], [175,60,50,80], [175,60,50,80], [175,60,50,80], [175,60,50,80], [175,55,50,80], 
    [175,55,50,85], [175,55,50,85], [175,60,50,85], [175,60,50,85], [175,60,50,85], [175,60,50,85], [175,60,50,85], 
    [175,60,50,85], [175,60,50,85], [175,60,50,85], [170,60,55,90], [170,60,55,90], [170,60,55,90], [170,60,55,90], 
    [170,60,55,95], [165,60,55,95], [165,60,60,95], [165,60,60,95], [165,60,65,95], [170,60,60,95], [170,55,65,100], 
    [170,55,65,100], [165,55,70,100], [165,55,65,105], [165,55,70,105], [160,50,70,110], [160,60,70,115], 
    [165,50,70,120], [165,50,70,120], [165,55,70,125], [165,55,75,125], [165,55,75,125], [165,55,80,125], 
    [165,50,80,130], [165,50,80,130], [165,50,80,130], [165,50,80,140], [160,45,90,145], [155,45,90,150], 
    [155,45,95,150], [160,40,90,155], [160,30,95,160], [160,30,105,165], [160,30,105,170], [160,30,105,175], 
    [160,30,105,180], [155,30,105,180], [150,30,105,180], [150,30,110,180], [150,25,120,190], [145,25,120,200], 
    [145,25,120,200], [145,15,125,210], [145,10,130,215], [140,5,135,220], [140,5,140,245], [140,5,145,245], 
    [140,0,150,260], [135,0,160,265], [125,0,165,300], [125,0,175,300], [125,0,175,300], [120,0,188,300]
]

# choose one of the above three sets of pixel coordinates and image lists:
aoi = sagehen_aoi;
images = sagehen_images;

### Set up options for main analysis routines:

In [ ]:
plot_figures = False
save_figures = False


# Apply radiometric correction?
# If we want to introduct a lag to our data correction, make sure to disable rad_correction
rad_correction = True
rad_correction_aoi = True # if we want to only look at the area of interest defined by the bounding boxes

# make empty array to hold the corrected data values
data_corrected = np.empty(data.shape);
hist_offset = np.empty(len(images));
snowpeak = np.empty(len(images));

# make empty arrays to hold the AOI mean, median, std values, altitude and px sizes
img_means_aoi = [];
img_medians_aoi = [];
img_stddevs_aoi = [];
img_max_aoi = [];
img_min_aoi = [];
img_range_aoi = [];
img_q95_aoi = [];

img_means_snow_aoi = [];
img_medians_snow_aoi = [];
img_stddevs_snow_aoi = [];
img_max_snow_aoi = [];
img_min_snow_aoi = [];
img_range_snow_aoi = [];

img_means_canopy_aoi = [];
img_medians_canopy_aoi = [];
img_stddevs_canopy_aoi = [];
img_max_canopy_aoi = [];
img_min_canopy_aoi = [];
img_range_canopy_aoi = [];

img_means_mixed_aoi = [];
img_medians_mixed_aoi = [];
img_stddevs_mixed_aoi = [];
img_max_mixed_aoi = [];
img_min_mixed_aoi = [];
img_range_mixed_aoi = [];


# I also want to keep track of the % of pixels that fall into three categories:
fraction_snow = []; # fraction (x100 to get %) of pixels that are most likely snow
n_snow = [];
fraction_canopy = []; # fraction (x100 to get %)  of pixels that are most likely forest canopy
n_canopy = [];
fraction_mixed = []; # fraction (x100 to get %)  of pixels that are likely a mixture of snow and canopy
n_mixed = []; # number of pixels that are likely a mixture of snow and canopy


# make empty arrays to hold pixel sizes
px_size = [];



buffer = 0; # AOI buffer (in pixels) if we want expand/shrink AOI size

# start some counters we'll need
data_counter = 0;
aoi_idx = 0;

In [ ]:
Tss = 0 # known, measured snow surface temperature that we will use to correct the TIR images
        # if Tss changes over time, this could be an array of Tss values
    
# set a temperature threshold to distinguish between cool (mostly snow) and warm (forest canopy) pixels:
low_threshold = 1 * np.ones((len(aoi))) # pixels below this will be classified as snow (about 1 C)
high_threshold = 10 * np.ones((len(aoi))) # pixels above this will be classified as canopy (about 10 C)

# If we want to introduct a lag to our data correction, make sure to disable rad_correction
#low_threshold = 1 - hist_bias
#high_threshold = 10 - hist_bias

### Run main analysis routines:

In [ ]:
for img in images:

    '''Radiometric correction using Tss:'''
    # Perform the radiometric correction
    if rad_correction == True:
        if rad_correction_aoi == True:
            # Get the original image data around the area of interest
            # This is used to perform the radiometric correction based on the AOI portion only
            image = data[aoi[aoi_idx][1]-buffer:aoi[aoi_idx][1]+aoi[aoi_idx][3]+buffer, \
                                  aoi[aoi_idx][0]-buffer:aoi[aoi_idx][0]+aoi[aoi_idx][2]+buffer, \
                                  img].reshape(-1);
        else:
            # Perform the correction based on the entire image
            image = data[:,:,img].reshape(-1)
        # Perform the radiometric correction of "data[:,:,img]",
        # based on the histogram of "image", (defined above)
        # and snow temperature reference point "Tss"
        data_corrected[:,:,img], hist_offset[img-len(images)], snowpeak[img-len(images)] = radiometric_correction(data[:,:,img],image,Tss)
    else:
        # Or don't apply the correction:
        data_corrected[:,:,img] = data[:,:,img]
        hist_offset[img-len(images)] = 0

    
    '''Calculate the temperature measures we'll use later:''' 
    # make the data_aoi from data_corrected (subsetting to just our area of interest)
    data_aoi = data_corrected[aoi[aoi_idx][1]-buffer:aoi[aoi_idx][1]+aoi[aoi_idx][3]+buffer, \
                          aoi[aoi_idx][0]-buffer:aoi[aoi_idx][0]+aoi[aoi_idx][2]+buffer, \
                          img];
    #data_aoi = data_corrected[:,:,img]; # ignore the aoi and use the entire image
    
    # count the total number of pixels in this AOI:
    n_pixels_aoi = count_px(data_aoi);
    
    # For the whole area of interest (AOI):
    [i_mean, i_median, i_std, i_max, i_min, i_range] = getSummaryStats(data_aoi)
    img_means_aoi.append(i_mean);
    img_medians_aoi.append(i_median);
    img_stddevs_aoi.append(i_std);
    img_max_aoi.append(i_max);
    img_min_aoi.append(i_min);
    img_range_aoi.append(i_range);
    img_q95_aoi.append(np.percentile(data_aoi,95))
    
    # For the snow pixels only (below low threshold):
    idx = (data_aoi<low_threshold[img-len(images)])
    [i_mean, i_median, i_std, i_max, i_min, i_range] = getSummaryStats(data_aoi[idx])
    img_means_snow_aoi.append(i_mean);
    img_medians_snow_aoi.append(i_median);
    img_stddevs_snow_aoi.append(i_std);
    img_max_snow_aoi.append(i_max);
    img_min_snow_aoi.append(i_min);
    img_range_snow_aoi.append(i_range);
    n_pixels_snow_aoi = count_px(data_aoi[idx]); # count the number of snow pixels in this AOI:
    n_snow.append(n_pixels_snow_aoi); # number of snow pixels
    fraction_snow.append(n_pixels_snow_aoi/n_pixels_aoi); # % of pixels that are most likely snow

    # For the canopy pixels: (two options -- include or exclude "mixed" pixels)
    idx = (data_aoi>low_threshold[img-len(images)]) # (above low threshold, this includes mixed pixels)
    #idx = (data_aoi>high_threshold[img-len(images)]) # (above high threshold, this does not include mixed pixels)
    [i_mean, i_median, i_std, i_max, i_min, i_range] = getSummaryStats(data_aoi[idx])
    img_means_canopy_aoi.append(i_mean);
    img_medians_canopy_aoi.append(i_median);
    img_stddevs_canopy_aoi.append(i_std);
    img_max_canopy_aoi.append(i_max);
    img_min_canopy_aoi.append(i_min);
    img_range_canopy_aoi.append(i_range);
    n_pixels_canopy_aoi = count_px(data_aoi[idx]); # count the number of canopy pixels in this AOI:
    n_canopy.append(n_pixels_canopy_aoi); # number of canopy pixels
    fraction_canopy.append(n_pixels_canopy_aoi/n_pixels_aoi); # % of pixels that are most likely snow
    
    # For the mixed pixels only (between thresholds):
    idx = (data_aoi>low_threshold[img-len(images)])*(data_aoi<high_threshold[img-len(images)])
    [i_mean, i_median, i_std, i_max, i_min, i_range] = getSummaryStats(data_aoi[idx])
    img_means_mixed_aoi.append(i_mean);
    img_medians_mixed_aoi.append(i_median);
    img_stddevs_mixed_aoi.append(i_std);
    img_max_mixed_aoi.append(i_max);
    img_min_mixed_aoi.append(i_min);
    img_range_mixed_aoi.append(i_range);
    n_pixels_mixed_aoi = count_px(data_aoi[idx]); # count the number of mixed pixels in this AOI:
    n_mixed.append(n_pixels_mixed_aoi); # number of mixed pixels
    fraction_mixed.append(n_pixels_mixed_aoi/n_pixels_aoi); # % of pixels that are most likely snow

    '''Calculate image resolution of each image:''' 
    # compute the image's approx. pixel size (GSD) (this assumes a flat surface below)
    img_width = (img_heights[img]*np.tan(np.radians(fov_x/2)))*2 # image width (in meters)
    img_height = (img_heights[img]*np.tan(np.radians(fov_y/2)))*2 # image height (in meters)
    # Calculate pixel size, the average of calculated pixel width and pixel height in meters
    px_size.append(((img_width/pix_x) + (img_height/pix_y)) / 2 );

    
    '''Plot figures:'''             
    if plot_figures == True:
        plt.figure(data_counter,figsize=(20,4))
              
        
        ### Plot histograms
        plt.subplot(131)
        n_bins = 50
        y_max = 50
        
        # plot full image histogram
        value, count = create_histogram(data_corrected[:,:,img],n_bins)
        percent_of_pixels = (count / (np.sum(count))) * 100 # calculate percentage of image from count of pixels
        plt.plot(value,percent_of_pixels,color='tab:gray',alpha=1,linewidth=1,linestyle='--',label='Corrected Image Histogram')
        
        # plot aoi  histogram
        value_aoi, count_aoi = create_histogram(data_aoi,n_bins)
        percent_of_pixels_aoi = (count_aoi / (np.sum(count_aoi))) * 100 # calculate percentage of image from count of pixels
        plt.plot(value_aoi,percent_of_pixels_aoi,color='tab:gray',alpha=1,linewidth=1,label='AOI Histogram')
        
        # plot a vertical line marking the AOI canopy mean temperature
        #plt.plot(np.linspace(img_means_canopy_aoi[aoi_idx],img_means_canopy_aoi[aoi_idx]),np.linspace(0,2),':g',label="AOI Mean $T_{F}$")
        plt.text(img_means_canopy_aoi[aoi_idx]+.5,1,"$\overline{T}_{F}$: " + str(np.round(img_means_canopy_aoi[aoi_idx],1)),color='g')
        
        # plot a vertical line for Tss
        #plt.plot(np.linspace(Tss,Tss),np.linspace(0,2),':b',label="Snow Surface Temp.")
        plt.text(.5,1,"$T_{SS}$: 0",color='b')
        
        # fill in histogram with colors showing the temperature classifications
        plt.fill_between(value_aoi, 0, percent_of_pixels_aoi, 
                         where=value_aoi<=np.ones_like(value_aoi)*1.25, 
                         facecolor='tab:blue', alpha=0.2,label='"snow" pixels') # < 1 C
        plt.fill_between(value_aoi, 0, percent_of_pixels_aoi, 
                         where=((value_aoi>=np.ones_like(value_aoi)*0.75) & (value_aoi<=np.ones_like(value_aoi)*10.25)), 
                         facecolor='tab:gray', alpha=0.2,label='"mixed" pixels') # 1 - 10 C
        plt.fill_between(value_aoi, 0, percent_of_pixels_aoi, 
                         where=value_aoi>=np.ones_like(value_aoi)*9.75, 
                         facecolor='tab:green', alpha=0.2,label='"canopy" pixels') # > 10 C
        
        # Plot formatting, labels
        plt.xlim([-5,25])
        plt.ylim([0,25])
        title_str = "Image: " + str(img) + ", Alt: " + str(img_heights[data_counter]) + " m AGL"
        plt.title('Temperature Histograms\n{}'.format(title_str))
        plt.xlabel('Temperature (C)')
        plt.ylabel('Percent of Image Pixels (%)')
        plt.legend(loc='upper right')


        ### Plot the full IR image
        ax = plt.subplot(132)
        plt.imshow(data_corrected[:,:,img],cmap='magma',vmin=0,vmax=20)
        cbar = plt.colorbar()
        cbar.ax.set_ylabel('Temperature (deg C)')
        frame1 = plt.gca()
        frame1.axes.get_xaxis().set_visible(False) # hide the x axis labels and tick marks
        frame1.axes.get_yaxis().set_visible(False) # hide the y axis labels and tick marks
        #title_str = " mean: " + str(np.round(np.mean(data_corrected[:,:,img]),2)) + " min: " + str(np.round(np.min(data_corrected[:,:,img]),2)) + " max: " + str(np.round(np.max(data_corrected[:,:,img]),2))
        plt.title(title_str)

        ### Plot the IR image AOI
        ax3 = plt.subplot(133)
        plt.imshow(data_aoi,vmin=0,vmax=20,cmap='magma')
        plt.title(str(aoi[aoi_idx][2]) +" x "+ str(aoi[aoi_idx][3]) + "px")
        frame2 = plt.gca()
        frame2.axes.get_xaxis().set_visible(False) # hide the x axis labels and tick marks
        frame2.axes.get_yaxis().set_visible(False) # hide the y axis labels and tick marks
        title_str = "AOI: Image Resolution: " + str(np.round(px_size[aoi_idx],2)) + " m \n"\
                    "mean: " + str(np.round(img_means_aoi[aoi_idx],2)) + \
                    ", min: " + str(np.round(img_min_aoi[aoi_idx],2)) + \
                    ", max: " + str(np.round(img_max_aoi[aoi_idx],2))
        plt.title(title_str)
        rect = patches.Rectangle((aoi[aoi_idx][0],aoi[aoi_idx][1]),aoi[aoi_idx][2],aoi[aoi_idx][3],\
                                 linewidth=1,edgecolor='w',facecolor='none')
        ax.add_patch(rect)
    
    
    aoi_idx = aoi_idx + 1;

    if save_figures == True:
        plt.savefig(r'plots\output\img_' + str(img) + '.png')
    data_counter = data_counter + 1;



In [ ]:
# Make a histogram of how far the measured snow temperature was from the expected Tss=0
hist_bias  = hist_offset[~np.isnan(hist_offset)]
rmse_tss, mean_bias_tss = rmse(Tss,hist_bias)
count, value = np.histogram(hist_bias, bins=20); 
new_value = [(i+value[idx+1])/2 for idx, i in enumerate(value) if idx < len(value)-1]
value = new_value
fig, (ax1, ax2) = plt.subplots(1,2,figsize=(10,3))
ax1.bar(value, count, width=0.2, color='k', edgecolor='k')
ax1.set_title('Temperature Biases Histogram')
ax1.set_ylabel('Number of Images');
ax1.set_xlabel('Temperature Bias (C)');

# Make a time series plot of the biases
ax2.plot(hist_bias,'.-k',markersize=10)
ax2.set_title('Temperature Biases over Time')
ax2.set_xlabel('Image Sequence Number');
ax2.set_ylabel('Temperature Bias (C)');

#plt.savefig('sagehen_biases.png', dpi=300)

***
***
### Look at the change in mixed pixel fraction vs image resolution:

### Plot change in fraction snow, canopy, mixed pixels:

In [ ]:
plt.figure(0,figsize=(7,4))
plt.plot(px_size,fraction_snow,'b.',marker='$*$',label="snow pixel fraction")
plt.plot(px_size,fraction_canopy,'g^',label="canopy pixel fraction")
plt.plot(px_size,fraction_mixed,'k.',markersize=10,label="mixed pixel fraction")
plt.ylim((0,1))
plt.xlabel('Image Resolution (pixel size, m)')
plt.ylabel('Fraction of Image')
plt.grid(b=True, which='major', color='grey', linestyle=':')
plt.title('Snow, Canopy, and Mixed Pixel Fractions')
plt.title('Fraction of image in each class')
plt.legend()
#plt.savefig(r'plots\PaperFigures\sagehen_allPixelFraction.png',
#            transparent=False, dpi=300)


print("Fraction Canopy Linear Regression")
quick_lin(px_size,fraction_canopy,'g')
print("Fraction Mixed Linear Regression")
quick_lin(px_size,fraction_mixed,'k')
print("Fraction snow Linear Regression")
quick_lin(px_size,fraction_snow,'b')
print("\nPixel sizes: ",px_size)

#### Mixed Pixel Fraction vs Image Resolution
(Figure S4a)

In [ ]:
plt.figure(0,figsize=(6,6))

mixed_fraction = np.array(fraction_mixed)/np.array(fraction_canopy)

plt.plot(px_size,mixed_fraction,'.',markersize=10,label="mixed pixel fraction",c='tab:blue')

# Linear regression:
[slope, intercept, r, p, SE] = stats.linregress(px_size,mixed_fraction)
print([slope, intercept, r, p, SE])
x = np.linspace(np.min(px_size),np.max(px_size),2)
plt.plot(x,x*slope + intercept,':',c='tab:blue')



#plt.ylim((0,1))
plt.xlabel('Image Resolution (pixel size, m)')
plt.ylabel('Fraction of Canopy Pixels')
#plt.grid(b=True, which='major', color='grey', linestyle=':')
#plt.title('Snow, Canopy, and Mixed Pixel Fractions')
plt.title('Mixed Pixel Fraction of Canopy')
#plt.legend()
###
plt.xlim((0,0.))
plt.xticks(np.arange(0, 0.21, 0.05))
plt.ylim((0,.4))
###
#plt.savefig(r'plots\davos2_mixedPixelFraction_.png',
#            transparent=True, dpi=300)





***
***
### Test for significant change in measured temperatures with image resolution:

### Select image set:

In [ ]:
# Select image set to use, assign mean, range, stddev values etc.

if sagehen == True:
    # Sagehen:
    idx = list(range(0,42))

if sagehen == False:
    #  Davos:
    idx = list(range(0,70)) # Davos 1 and 2

### Select temperature class to test:
(Figure S4b) (options: all, canopy, snow, mixed)

#### Test Tcanopy:

In [ ]:
# Assign Tcanopy values:
means = np.array(img_means_canopy_aoi[idx[0]:idx[-1]]);
mins = np.array(img_min_canopy_aoi[idx[0]:idx[-1]]);
maxs = np.array(img_max_canopy_aoi[idx[0]:idx[-1]]);
stddevs = np.array(img_stddevs_canopy_aoi[idx[0]:idx[-1]]);
px_sizes = px_size[idx[0]:idx[-1]]

n = len(means) # number of image samples we have
std_error = 0 # relative error between pixels w/in a single image
std_error_mean = 1 # absolute error between individual images of the same scene

test_for_T_change(px_sizes,means,stddevs,mins,maxs,std_error_mean,std_error,'tab:blue',True)

In [ ]:
### Monte Carlo tests on the significance of measurement error to linear regression results:
s = 1 # introduce a measurement error normally distributed within +/1 deg C
m = 100 # number of test runs to perform

# If i'm doing a 2-sided test with alpha=5%, then reject the null (no slope) if p <= alpha/2
msmtErrorTest_plots(px_sizes,means,s,m)

In [ ]:
# plot mean vs mixed canopy fraction
plt.plot(fraction_mixed[0:],means,'.k')
plt.ylabel('mean')
plt.xlabel('fmixed')
quick_lin(fraction_mixed[0:],means)

#### Test Tsnow:

In [ ]:
# Assign Tsnow values:
means = np.array(img_means_snow_aoi[idx[0]:idx[-1]]);
mins= np.array(img_min_snow_aoi[idx[0]:idx[-1]]);
maxs= np.array(img_max_snow_aoi[idx[0]:idx[-1]]);
stddevs = np.array(img_stddevs_snow_aoi[idx[0]:idx[-1]]);
px_sizes = px_size[idx[0]:idx[-1]]

n = len(means) # number of image samples we have
std_error = 0 # relative error between pixels w/in a single image
std_error_mean = 1 # absolute error between individual images of the same scene

test_for_T_change(px_sizes,means,stddevs,mins,maxs,std_error_mean,std_error)

In [ ]:
### Monte Carlo tests on the significance of measurement error to linear regression results:
s = 1 # measurement error in +/1 deg C
m = 100 # number of test runs with normally distributed random error to perform

# If i'm doing a 2-sided test with alpha=5%, then reject the null (no slope) if p <= alpha/2
msmtErrorTest_plots(px_sizes,means,s,m)

### Test Tmixed:

In [ ]:
# Assign Tmixed values:
means = np.array(img_means_mixed_aoi[idx[0]:idx[-1]]);
mins = np.array(img_min_mixed_aoi[idx[0]:idx[-1]]);
maxs = np.array(img_max_mixed_aoi[idx[0]:idx[-1]]);
stddevs = np.array(img_stddevs_mixed_aoi[idx[0]:idx[-1]]);
px_sizes = px_size[idx[0]:idx[-1]]

n = len(means) # number of image samples we have
std_error = 0 # relative error between pixels w/in a single image
std_error_mean = 1 # absolute error between individual images of the same scene

test_for_T_change(px_sizes,means,stddevs,mins,maxs,std_error_mean,std_error)

In [ ]:
### Monte Carlo tests on the significance of measurement error to linear regression results:
s = 1 # measurement error in +/1 deg C
m = 100 # number of test runs with normally distributed random error to perform

# If i'm doing a 2-sided test with alpha=5%, then reject the null (no slope) if p <= alpha/2
msmtErrorTest_plots(px_sizes,means,s,m)

#### Test all temperatures within the area of interest:

In [ ]:
# Assign all AOI T values:
means = np.array(img_means_aoi[idx[0]:idx[-1]]);
mins = np.array(img_min_aoi[idx[0]:idx[-1]]);
maxs = np.array(img_max_aoi[idx[0]:idx[-1]]);
stddevs = np.array(img_stddevs_aoi[idx[0]:idx[-1]]);
px_sizes = px_size[idx[0]:idx[-1]]

n = len(means) # number of image samples we have
std_error = 0 # relative error between pixels w/in a single image
std_error_mean = 1 # absolute error between individual images of the same scene

test_for_T_change(px_sizes,means,stddevs,mins,maxs,std_error_mean,std_error)

In [ ]:
### Monte Carlo tests on the significance of measurement error to linear regression results:
s = 1 # measurement error in +/1 deg C
m = 100 # number of test runs with normally distributed random error to perform

# If i'm doing a 2-sided test with alpha=5%, then reject the null (no slope) if p <= alpha/2
msmtErrorTest_plots(px_sizes,maxs,s,m)

***
***
#### Retrieving surface temperatures from selected images at Sagehen:

In [ ]:
# get a horizontal cross section across a couple images to compre before and after data correction

n = 363; # the row we want to draw a transect line across

for i in [5,42]: # look at a couple images (image #5, and #42)
    fig, (ax_img, ax_temp) = plt.subplots(1,2,figsize=(14,5))
    
    # show the image itself
    ax_img.imshow(data_corrected[:,:,i],cmap='magma',vmin=-2,vmax=20,origin='upper');
    ax_img.plot(list(range(0,640)),np.ones(len(list(range(0,640))))*n,':w',label='Transect Line')
    ax_img.legend(loc='upper left')
    ax_img.set_xlabel('Image Pixel Column')
    ax_img.set_ylabel('Image Pixel Row')
    ax_img.set_title('TIR Image #{}'.format(i))
    
    # show a temperature transect across the image
    ax_temp.plot(data[n,:,i],'--',c='tab:gray',label='Original Surface Temp.');
    ax_temp.plot(data_corrected[n,:,i],'-k',label='Corrected Surface Temp.');
    ax_temp.plot(list(range(0,640)),np.zeros(len(list(range(0,640)))),':k',label='$T_{ss}$ = 0 $\degree$C')
    ax_temp.set_ylabel('Surface Temperature ($\degree$C)')
    ax_temp.set_xlabel('Image Pixel Column')
    ax_temp.set_xlim((0,640))
    ax_temp.set_title('Temperature Along a Transect Line')
    ax_temp.legend()

***
### Sagehen: What is the water surface temperature of Sagehen Creek?

In [ ]:
plot_images = False

# image pixel coordinates of a visible portion of Sagehen Creek
x = [150,150,160,160,175,190,180,180,190,190,200,210,210,210,210,220,220,210,210,210,220,220,210]
y = [410,420,450,450,450,420,430,430,420,430,410,420,420,420,420,400,400,400,400,400,390,400,390]

w = 15; # window size in px to grab water temperature from within
water_temps = [];
water_temps_nc = [];

for i in range(22,45):
    idx = i-22;
    if plot_images == True:
        plt.figure(idx,figsize=(12,8))
        plt.imshow(data_corrected[:,:,i],cmap='magma',vmin=-2,vmax=20);
        plt.colorbar()
        plt.tick_params(axis='both', left=False, bottom=False, labelleft=False, labelbottom=False)
        plt.plot(x[idx],y[idx],'.w',label='Sagehen Creek AOI')
        plt.plot(x[idx]+w,y[idx],'|w')
        plt.plot(x[idx],y[idx]+w,'_w')
        plt.plot(x[idx],y[idx]-w,'_w')
        plt.plot(x[idx]-w,y[idx],'|w')
        plt.legend(loc='lower right')

    water_temps.append(data_corrected[y[idx]-w:y[idx]+w,x[idx]-w:x[idx]+w,i])
    water_temps_nc.append(data[y[idx]-w:y[idx]+w,x[idx]-w:x[idx]+w,i])
    
    
cleanhist(water_temps,n_bins=500)
print("Standard Deviation: " + str(np.round(np.std(water_temps),2)))
plt.xlim((3,7))
#plt.savefig(r'\plots\\'+str(i)+'_hist.png',
#            transparent=False, dpi=300)

print('points',len(x))
# Calculate RMSE against USGS gage water temperature:
tw = 4.6 # mean USGS gage water temperature @ 15:00 - 16:00
tw_rmse, mean_bias = rmse(tw, np.array(water_temps))
print('RMSE:',tw_rmse)
print('Mean Bias:',mean_bias)

***
### Sagehen: What is the snow surface temperature within this meadow?
We can pick out a small area from each image:

In [ ]:
plot_images = False

# image pixel coordinates of a poriton of snow-covered meadow
x = [150, 150, 150, 150, 150, 200, 200, 200, 180, 190, 200, 200, 200, 200, 190, 210, 190,
     550, 550, 550, 550, 550, 550, 550, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500,
     500, 500, 500, 500, 500, 500]
y = [480, 480, 460, 460, 470, 450, 440, 420, 320, 310, 310, 300, 290, 290, 280, 300, 230, 
     300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300,
     300, 300, 250, 250, 250, 250]

w = 30; # window size in px to grab meadow snow temperature from within
meadow_snow_temps = []; # to hold the corrected data
meadow_snow_temps_nc = []; # to hold data that is "not corrected"

for i in range(5,45):
    idx = i-5;
    if plot_images == True:
        plt.figure(idx,figsize=(8,5))
        plt.imshow(data_corrected[:,:,i],cmap='magma',vmin=-2,vmax=20);
        plt.colorbar()
        #plt.tick_params(axis='both', left=False, bottom=False, labelleft=False, labelbottom=False)
        plt.plot(x[idx],y[idx],'.w',label='Sagehen Creek AOI')
        plt.plot(x[idx]+w,y[idx],'|w')
        plt.plot(x[idx],y[idx]+w,'_w')
        plt.plot(x[idx],y[idx]-w,'_w')
        plt.plot(x[idx]-w,y[idx],'|w')
        plt.legend(loc='lower right')

    meadow_snow_temps.append(data_corrected[y[idx]-w:y[idx]+w,x[idx]-w:x[idx]+w,i])
    meadow_snow_temps_nc.append(data[y[idx]-w:y[idx]+w,x[idx]-w:x[idx]+w,i])
    
    
cleanhist(meadow_snow_temps,n_bins=200)
print("Standard Deviation: " + str(np.round(np.std(meadow_snow_temps),2)))
plt.xlim((-2,2))
#plt.savefig(r'plots\\'+str(i)+'_hist.png',
#            transparent=False, dpi=300)

print('points',len(x))
# Calculate RMSE against our melting Tss=0
tss = 0
tss_rmse, mean_bias = rmse(tss, np.array(meadow_snow_temps))
print('RMSE:',tss_rmse)
print('Mean Bias:',mean_bias)

Or sample along a transect in a single image:

In [ ]:
# get a horizontal cross section across a couple images to compre before and after data correction

x = np.linspace(240,520,50);
y = np.linspace(10,480,50);
w = 10

for i in [42]: # look at a couple images (image #5, and #42)
    fig, (ax_img, ax_temp) = plt.subplots(1,2,figsize=(14,5))
    
    # show the image itself
    ax_img.imshow(data_corrected[:,:,i],cmap='magma',vmin=-2,vmax=20,origin='upper');
    ax_img.plot(x,y,':w',label='Transect Line')
    ax_img.legend(loc='upper left')
    ax_img.set_xlabel('Image Pixel Column')
    ax_img.set_ylabel('Image Pixel Row')
    ax_img.set_title('TIR Image #{}'.format(i))
    
    # get the temperature across the transect line
    snow_temps_nc = [];
    snow_temps = [];
    for ii in range(0,x.shape[0]):
        snow_temps_nc.append(data[int(y[ii])-w:int(y[ii])+w,int(x[ii])-w:int(x[ii])+w,i])
        snow_temps.append(data_corrected[int(y[ii])-w:int(y[ii])+w,int(x[ii])-w:int(x[ii])+w,i])
    snow_temps_nc = np.array(snow_temps_nc).ravel()
    snow_temps = np.array(snow_temps).ravel()
    
    # show a temperature transect across the image
    ax_temp.plot(snow_temps_nc,'--',c='tab:gray',label='Original Surface Temp.');
    ax_temp.plot(snow_temps,'-k',alpha=1,label='Corrected Surface Temp.');
    ax_temp.plot([0,snow_temps.shape[0]],[0,0],':k',label='$T_{ss}$ = 0 $\degree$C')
    ax_temp.set_ylabel('Surface Temperature ($\degree$C)')
    ax_temp.set_xlabel('Sample Count')
    ax_temp.set_ylim((-5,25))
    ax_temp.set_title('Temperature Along a Transect Line')
    ax_temp.legend()
    
# Plot a histogram
cleanhist(snow_temps,n_bins=50)
plt.title('Transect Line Temperature Histogram')
plt.xlim((-3,3))

#plt.savefig(r'plots\\'+str(i)+'_hist.png',
#            transparent=False, dpi=300)


print("Standard Deviation: " + str(np.round(np.std(snow_temps),2)))
# Calculate RMSE against mean fixed radiometer snow surface temperature:
tss = 0
tss_rmse, mean_bias = rmse(tss, np.array(snow_temps))
print('RMSE:',tss_rmse)
print('Mean Bias:',mean_bias)

***
#### What are some other forest canopy temperatures around here?

In [ ]:
# get a horizontal cross section across a couple images to compre before and after data correction

x = np.linspace(240,150,50);
y = np.linspace(170,340,50);
w = 1

for i in [42]: # look at a couple images (image #5, and #42)
    fig, (ax_img, ax_temp) = plt.subplots(1,2,figsize=(14,5))
    
    # show the image itself
    ax_img.imshow(data_corrected[:,:,i],cmap='magma',vmin=-2,vmax=20,origin='upper');
    ax_img.plot(x,y,':w',label='Transect Line')
    ax_img.legend(loc='upper left')
    ax_img.set_xlabel('Image Pixel Column')
    ax_img.set_ylabel('Image Pixel Row')
    ax_img.set_title('TIR Image #{}'.format(i))
    
    # get the temperature across the transect line
    snow_temps_nc = [];
    snow_temps = [];
    for ii in range(0,x.shape[0]):
        snow_temps_nc.append(data[int(y[ii])-w:int(y[ii])+w,int(x[ii])-w:int(x[ii])+w,i])
        snow_temps.append(data_corrected[int(y[ii])-w:int(y[ii])+w,int(x[ii])-w:int(x[ii])+w,i])
    snow_temps_nc = np.array(snow_temps_nc).ravel()
    snow_temps = np.array(snow_temps).ravel()
    
    # show a temperature transect across the image
    ax_temp.plot(snow_temps_nc,'--',c='tab:gray',label='Original Surface Temp.');
    ax_temp.plot(snow_temps,'-k',alpha=1,label='Corrected Surface Temp.');
    ax_temp.plot([0,snow_temps.shape[0]],[0,0],':k',label='$T_{ss}$ = 0 $\degree$C')
    ax_temp.set_ylabel('Surface Temperature ($\degree$C)')
    ax_temp.set_xlabel('Sample Count')
    ax_temp.set_ylim((-5,25))
    ax_temp.set_title('Temperature Along a Transect Line')
    ax_temp.legend()
    
# Plot a histogram
cleanhist(snow_temps,n_bins=100)
plt.title('Transect Line Temperature Histogram')
#plt.xlim((-3,3))

#plt.savefig(r'plots\\'+str(i)+'_hist.png',
#            transparent=False, dpi=300)


print("Standard Deviation: " + str(np.round(np.std(snow_temps),2)))
# Calculate RMSE against mean fixed radiometer Tf:
tss = 16.9
tss_rmse, mean_bias = rmse(tss, np.array(snow_temps))
print('RMSE:',tss_rmse)
print('Mean Bias:',mean_bias)

#### What is the canopy temperature of the instrumented tree?
... at the highest resolution available from the drone imagery

In [ ]:
# get a horizontal cross section across a couple images to compre before and after data correction

x = np.linspace(150,100,50);
y = np.linspace(300,350,50);
w = 1

for i in [5]: # look at a couple images (image #5, and #42)
    fig, (ax_img, ax_temp) = plt.subplots(1,2,figsize=(14,5))
    
    # show the image itself
    ax_img.imshow(data_corrected[:,:,i],cmap='magma',vmin=-2,vmax=20,origin='upper');
    ax_img.plot(x,y,':w',label='Transect Line')
    ax_img.legend(loc='upper left')
    ax_img.set_xlabel('Image Pixel Column')
    ax_img.set_ylabel('Image Pixel Row')
    ax_img.set_title('TIR Image #{}'.format(i))
    
    # get the temperature across the transect line
    snow_temps_nc = [];
    snow_temps = [];
    for ii in range(0,x.shape[0]):
        snow_temps_nc.append(data[int(y[ii])-w:int(y[ii])+w,int(x[ii])-w:int(x[ii])+w,i])
        snow_temps.append(data_corrected[int(y[ii])-w:int(y[ii])+w,int(x[ii])-w:int(x[ii])+w,i])
    snow_temps_nc = np.array(snow_temps_nc).ravel()
    snow_temps = np.array(snow_temps).ravel()
    
    # show a temperature transect across the image
    ax_temp.plot(snow_temps_nc,'--',c='tab:gray',label='Original Surface Temp.');
    ax_temp.plot(snow_temps,'-k',alpha=1,label='Corrected Surface Temp.');
    ax_temp.plot([0,snow_temps.shape[0]],[0,0],':k',label='$T_{ss}$ = 0 $\degree$C')
    ax_temp.set_ylabel('Surface Temperature ($\degree$C)')
    ax_temp.set_xlabel('Sample Count')
    ax_temp.set_ylim((-2,30))
    ax_temp.set_title('Temperature Along a Transect Line')
    ax_temp.legend()
    
# Plot a histogram
cleanhist(snow_temps,n_bins=50)
plt.title('Transect Line Temperature Histogram')
#plt.xlim((-3,3))

#plt.savefig(r'plots\\'+str(i)+'_hist.png',
#            transparent=False, dpi=300)


print("Standard Deviation: " + str(np.round(np.std(snow_temps),2)))
# Calculate RMSE against mean fixed radiometer Tf:
tss = 16.9
tss_rmse, mean_bias = rmse(tss, np.array(snow_temps))
print('RMSE:',tss_rmse)
print('Mean Bias:',mean_bias)

At a lower resolution...

In [ ]:
# get a horizontal cross section across a couple images to compre before and after data correction

x = np.linspace(470,490,50);
y = np.linspace(200,180,50);
w = 1

for i in [42]: # look at a couple images (image #5, and #42)
    fig, (ax_img, ax_temp) = plt.subplots(1,2,figsize=(14,5))
    
    # show the image itself
    ax_img.imshow(data_corrected[:,:,i],cmap='magma',vmin=-2,vmax=20,origin='upper');
    ax_img.plot(x,y,':w',label='Transect Line')
    ax_img.legend(loc='upper left')
    ax_img.set_xlabel('Image Pixel Column')
    ax_img.set_ylabel('Image Pixel Row')
    ax_img.set_title('TIR Image #{}'.format(i))
    
    # get the temperature across the transect line
    snow_temps_nc = [];
    snow_temps = [];
    for ii in range(0,x.shape[0]):
        snow_temps_nc.append(data[int(y[ii])-w:int(y[ii])+w,int(x[ii])-w:int(x[ii])+w,i])
        snow_temps.append(data_corrected[int(y[ii])-w:int(y[ii])+w,int(x[ii])-w:int(x[ii])+w,i])
    snow_temps_nc = np.array(snow_temps_nc).ravel()
    snow_temps = np.array(snow_temps).ravel()
    
    # show a temperature transect across the image
    ax_temp.plot(snow_temps_nc,'--',c='tab:gray',label='Original Surface Temp.');
    ax_temp.plot(snow_temps,'-k',alpha=1,label='Corrected Surface Temp.');
    ax_temp.plot([0,snow_temps.shape[0]],[0,0],':k',label='$T_{ss}$ = 0 $\degree$C')
    ax_temp.set_ylabel('Surface Temperature ($\degree$C)')
    ax_temp.set_xlabel('Sample Count')
    ax_temp.set_ylim((-5,25))
    ax_temp.set_title('Temperature Along a Transect Line')
    ax_temp.legend()
    
# Plot a histogram
cleanhist(snow_temps,n_bins=20)
plt.title('Transect Line Temperature Histogram')
#plt.xlim((-3,3))

#plt.savefig(r'plots\\'+str(i)+'_hist.png',
#            transparent=False, dpi=300)


print("Standard Deviation: " + str(np.round(np.std(snow_temps),2)))
# Calculate RMSE against mean fixed radiometer Tf:
tss = 16.9
tss_rmse, mean_bias = rmse(tss, np.array(snow_temps))
print('RMSE:',tss_rmse)
print('Mean Bias:',mean_bias)

***
***
### Image Upscaling (Aggregation):

In [ ]:
### Resample all images to a target resolution (matching that of the aircraft)
### Use these to compare mean and standard deviation against the aircraft images
### mean and standard deviations (for full image, or AOI only)

target_res = 1.0 # target resolution in meters, approximate
buffer=0
plot_images = False;
save_images = False;

original_means = [];
original_stddevs = [];
downscaled_means = [];
downscaled_stddevs = [];

for im_index in range(0,len(images)):
    
    im_num = images[im_index]
    
    # Compute these based on entire image:
    #data_aoi = data_corrected[:,:,im_num];
    
    # Option to compute these based on the AOI only:
    buffer = 10
    data_aoi = data_corrected[aoi[im_index][1]-buffer:aoi[im_index][1]+aoi[im_index][3]+buffer, \
                              aoi[im_index][0]-buffer:aoi[im_index][0]+aoi[im_index][2]+buffer, \
                              im_num]
    
    
    original_means.append(np.mean(data_aoi));
    original_stddevs.append(np.std(data_aoi));
    
    
    if plot_images == True:
        plt.figure(im_index+10,figsize=(18,5))
        plt.subplot(1,2,1)
        plt.imshow(data_aoi,cmap='magma',vmin=0,vmax=20)
        plt.title('Original Image ' + str(np.round(px_size[im_index],2)) + 'm/px')
        plt.subplot(1,2,2)
        plt.imshow(lower_res,cmap='magma',vmin=0,vmax=20)
        plt.title('Gaussian Upscaled to ' + str(np.round(new_res,2)) + 'm/px')
        plt.colorbar()
        if save_images == True:
            plt.savefig(r'plots\resampling\images' + str(im_index) + ".png")
    
    ##### use OpenCV for gaussian pyramid:
    steps = int(np.round(np.log(target_res/px_size[im_index])/(np.log(2))))
    new_res = px_size[im_index] * 2**steps
    lower_res = data_aoi
    for i in range(0,steps):
        lower_res = cv2.pyrDown(lower_res)

    downscaled_means.append(np.mean(lower_res));
    downscaled_stddevs.append(np.std(lower_res));
    

# plot a sample image pair
plt.figure(5,figsize=(10,4))
plt.subplot(121)
plt.imshow(data_aoi,cmap='magma',vmin=0, vmax=20)
plt.title("Original Image")
plt.colorbar()
plt.subplot(122)
plt.imshow(lower_res,cmap='magma',vmin=0, vmax=20)
plt.title("Upscaled Image")
plt.colorbar()
plt.tight_layout()
    
# compare histograms of image means before and after downscaling
plt.figure(4,figsize=(7,7))
plt.subplot(221)
count, value = np.histogram(original_means); 
# take the bin edges (value) and change them to bin centers
# so that number of "count" = number of "value"
new_value = [(i+value[idx+1])/2 for idx, i in enumerate(value) if idx < len(value)-1]
value = new_value
plt.plot(value,count,'-k', label='original means')
count, value = np.histogram(downscaled_means); 
# take the bin edges (value) and change them to bin centers
# so that number of "count" = number of "value"
new_value = [(i+value[idx+1])/2 for idx, i in enumerate(value) if idx < len(value)-1]
value = new_value
plt.plot(value,count,'--k',label='upscaled means')
plt.legend()

# compare histograms of image standard deviations before and after downscaling
plt.subplot(222)
count, value = np.histogram(original_stddevs); 
# take the bin edges (value) and change them to bin centers
# so that number of "count" = number of "value"
new_value = [(i+value[idx+1])/2 for idx, i in enumerate(value) if idx < len(value)-1]
value = new_value
plt.plot(value,count,'-k', label='original stddevs')
count, value = np.histogram(downscaled_stddevs); 
# take the bin edges (value) and change them to bin centers
# so that number of "count" = number of "value"
new_value = [(i+value[idx+1])/2 for idx, i in enumerate(value) if idx < len(value)-1]
value = new_value
plt.plot(value,count,'--k',label='upscaled stddevs')
plt.legend()
    
# look at how the original mean compares with downscaled image mean
plt.subplot(223)
plt.scatter(original_means,downscaled_means,s=2,c='k')
plt.plot(np.linspace(np.min(original_means)-1,np.max(original_means)+1,10),np.linspace(np.min(original_means)-1,np.max(original_means)+1,10),':b')
plt.xlabel('Original Mean (C)')
plt.ylabel('Gaussian Upscaled Mean (C)')
plt.title('Image Means')

# look at how the original stddev compares with downscaled image stddev
plt.subplot(224)
plt.scatter(original_stddevs,downscaled_stddevs,s=2,c='k')
plt.plot(np.linspace(np.min(original_stddevs)-1,np.max(original_stddevs)+1,10),np.linspace(np.min(original_stddevs)-1,np.max(original_stddevs)+1,10),':b')
plt.xlabel('Original Standard Deviation (C)')
plt.ylabel('Gaussian Upscaled Standard Deviation (C)')
plt.title('Image Standard Deviations')
plt.tight_layout()